# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

**Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.**

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

## 1. Data Understanding

In [2]:
def read_data(df):
    
    '''
    Print out shape, data types, number of nulls for each
    '''   
    num_entries = len(df)
    num_col = df.shape[1]
    col_type = df.dtypes
    col_null = df.isnull().sum()
    
    print('There are '+str(num_entries) +' entries')
    print('There are '+str(num_col) +' columns')
    print(' ')
    print('These are the data types --')
    print(col_type)
    print(' ')
    print('The columns have the following number of nulls')
    print(col_null)
    print(' ')
    print(' ')
    
    return df.head()

###### Portfolio data

In [3]:
read_data(portfolio)

There are 10 entries
There are 6 columns
 
These are the data types --
channels      object
difficulty     int64
duration       int64
id            object
offer_type    object
reward         int64
dtype: object
 
The columns have the following number of nulls
channels      0
difficulty    0
duration      0
id            0
offer_type    0
reward        0
dtype: int64
 
 


,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5


In [93]:
portfolio

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2


In [4]:
# How many different channels are there?
portfolio['channels'].apply(lambda x:len(x)).max()

4

In [16]:
# number of unique ids
portfolio['id'].unique()

array(['ae264e3637204a6fb9bb56bc8210ddfd',
       '4d5c57ea9a6940dd891ad53e9dbe8da0',
       '3f207df678b143eea3cee63160fa8bed',
       '9b98b8c7a33c4b65b9aebfe6a799e6d9',
       '0b1e1539f2cc45b7b9fa7c272da2e1d7',
       '2298d6c36e964ae4a3e7e9706d1fb8c2',
       'fafdcd668e3743c1bb461111dcafc2a4',
       '5a8bc65990b245e5a138643cd4eb9837',
       'f19421c1d4aa40978ebb69ca19b0e20d',
       '2906b810c7d4411798c6938adc9daaa5'], dtype=object)

In [5]:
# Which are the offer types?
portfolio['offer_type'].unique()

array(['bogo', 'informational', 'discount'], dtype=object)

In [6]:
# Do informational offers come with any rewards?
portfolio[portfolio['offer_type']=='informational']['reward'].max()

0

###### Profile data

In [7]:
read_data(profile)

There are 17000 entries
There are 5 columns
 
These are the data types --
age                   int64
became_member_on      int64
gender               object
id                   object
income              float64
dtype: object
 
The columns have the following number of nulls
age                    0
became_member_on       0
gender              2175
id                     0
income              2175
dtype: int64
 
 


,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [8]:
print('Earliest joining date: ', profile['became_member_on'].min())
print('Latest joining date: ', profile['became_member_on'].max())

Earliest joining date:  20130729
Latest joining date:  20180726


###### Transcript data

In [9]:
read_data(transcript)

There are 306534 entries
There are 4 columns
 
These are the data types --
event     object
person    object
time       int64
value     object
dtype: object
 
The columns have the following number of nulls
event     0
person    0
time      0
value     0
dtype: int64
 
 


,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


In [10]:
print('Max time: ', transcript['time'].max())

Max time:  714


In [108]:
transcript['event'].value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

In [11]:
print('types of events:')
transcript['event'].unique()

types of events:


array(['offer received', 'offer viewed', 'transaction', 'offer completed'],
      dtype=object)

In [89]:
# Number of unique customers in transaction data
len(set(transcript[transcript['event']=='offer received']['person']))

16994

In [91]:
# Number of unique control customers (who have not received offers if there are any at all)
len(set(transcript[transcript['event']=='transaction']['person']) - \
    set(transcript[transcript['event']=='offer received']['person']))

6

Control group is almost non-existent in the data. More than 99% of the customers in the data have received offers.

In [12]:
# Examine the events of customer #1
transcript[transcript['person']=='78afa995795e4d85b5d9ceeca43f5fef']

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
15561,offer viewed,78afa995795e4d85b5d9ceeca43f5fef,6,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
47582,transaction,78afa995795e4d85b5d9ceeca43f5fef,132,{'amount': 19.89}
47583,offer completed,78afa995795e4d85b5d9ceeca43f5fef,132,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...
49502,transaction,78afa995795e4d85b5d9ceeca43f5fef,144,{'amount': 17.78}
53176,offer received,78afa995795e4d85b5d9ceeca43f5fef,168,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
85291,offer viewed,78afa995795e4d85b5d9ceeca43f5fef,216,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
87134,transaction,78afa995795e4d85b5d9ceeca43f5fef,222,{'amount': 19.67}
92104,transaction,78afa995795e4d85b5d9ceeca43f5fef,240,{'amount': 29.72}
141566,transaction,78afa995795e4d85b5d9ceeca43f5fef,378,{'amount': 23.93}


In [13]:
# Why are some offers never 'completed' even though a transaction has occured after offer is sent and viewed?
portfolio[portfolio['id']=='5a8bc65990b245e5a138643cd4eb9837']

,channels,difficulty,duration,id,offer_type,reward
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0


In [14]:
# Examine the events of customer #2
transcript[transcript['person']=='a03223e636434f42ac4c3df47e8bac43']

,event,person,time,value
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
15562,offer viewed,a03223e636434f42ac4c3df47e8bac43,6,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
90553,transaction,a03223e636434f42ac4c3df47e8bac43,234,{'amount': 1.09}
97700,transaction,a03223e636434f42ac4c3df47e8bac43,264,{'amount': 3.5}
110829,offer received,a03223e636434f42ac4c3df47e8bac43,336,{'offer id': '3f207df678b143eea3cee63160fa8bed'}
123539,offer viewed,a03223e636434f42ac4c3df47e8bac43,336,{'offer id': '3f207df678b143eea3cee63160fa8bed'}
150599,offer received,a03223e636434f42ac4c3df47e8bac43,408,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
201573,offer received,a03223e636434f42ac4c3df47e8bac43,504,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
245125,offer received,a03223e636434f42ac4c3df47e8bac43,576,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
277104,transaction,a03223e636434f42ac4c3df47e8bac43,612,{'amount': 0.06}


The following data entry rules are observed from the transcript data:
- Informational offers do not come with any rewards, hence they cannot be completed even when it has been viewed a transaction occurs within its duration.
- Offers do not have to be viewed to be completed.
- The data also contains transactions that are not associated with any offers.
- A single transaction can complete two offers.

Therefore, the following assumptions are to be taken for the subsequent analysis.
- If a transaction occured after an informational offer is viewed, within the effective duration, it is attributed to the offer.
- If there are multiple transactions attributed a single type of offer per unique customer, it will be attributed to the last received offer.
- A single transaction can also be attributed to multiple offers, as long as the customer has viewed both and the transaction is within effective duration.
- If an offer is not viewed before it is completed, then the transaction is not attributed to the offer.

## 2. Data Preprocessing

Before further analysis, we need to carry out steps to clean and transform the data. The goal is to get to a unique person-offer or person-transaction (if transaction is not linked to any offer) per line, with all customer and offer data combined on a single dataframe. These are some of the steps that I would do:
- Impute nulls
- Convert data types
- Transpose and summarise data
- Join datasets

###### Transcript data
The data will be transposed such that one offer is on one row, showing whether the offer has been received, viewed, transacted and completed

In [36]:
# Parse value column - some of the offer id keys are entered as 'order_id'
value_parse_col = ['offer id', 'offer_id', 'amount']

for col in value_parse_col:
    transcript[col] = transcript['value'].apply(lambda x:x[col] if col in x.keys() else '')

In [46]:
# How many offer id's are under the 'offer_id' column?
(transcript['offer_id'].apply(lambda x:len(x))>1).sum()

33579

In [52]:
# Combine the offer id columns
transcript['clean_offer_id'] = transcript["offer_id"].map(str) + transcript["offer id"]
transcript = transcript.drop(['offer id', 'offer_id'], axis = 1)
transcript.head()

,event,person,time,value,amount,clean_offer_id
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},,2906b810c7d4411798c6938adc9daaa5
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},,fafdcd668e3743c1bb461111dcafc2a4
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},,4d5c57ea9a6940dd891ad53e9dbe8da0


In [58]:
transcript_received = transcript[transcript['event']=='offer received'][['person', 'clean_offer_id', 'time']]
transcript_viewed = transcript[transcript['event']=='offer viewed'][['person', 'clean_offer_id', 'time']]
transcript_completed = transcript[transcript['event']=='offer completed'][['person', 'clean_offer_id', 'time']]
transcript_transact = transcript[transcript['event']=='transaction'][['person', 'amount', 'time']]

In [78]:
transcript_new = transcript_received.merge(transcript_viewed, how = 'outer', on = ['clean_offer_id', 'person'], \
                          suffixes = ('_rec', '_view'))

#transcript_new = transcript_new.merge(transcript_completed, how = 'outer', on = ['clean_offer_id', 'person'])
#transcript_new = transcript_new.rename(columns={'time': 'time_comp'})

print('Total num of offers sent:', len(transcript_received))
print('Length of df:', len(transcript_new))
transcript_new.head(20)

Total num of offers sent: 76277
Length of df: 95321


,person,clean_offer_id,time_rec,time_view
0,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,6.0
1,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,6.0
2,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,624.0
3,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,504,6.0
4,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,504,624.0


There are duplicate rows after merge which have to be eliminated. There are obvious ones that do not make sense - offers are viewed before they are received, completed before they are viewed.

In [119]:
transcript_new.head(30)

,person,clean_offer_id,time_rec,time_view
0,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,6.0
1,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,6.0
2,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,624.0
3,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,504,6.0
4,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,504,624.0
5,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,576,6.0
6,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,576,624.0
7,e2127556f4f64592b11af22de27a7932,2906b810c7d4411798c6938adc9daaa5,0,18.0
8,8ec6ce2a7e7949b1bf142def7d0e0586,fafdcd668e3743c1bb461111dcafc2a4,0,12.0
9,8ec6ce2a7e7949b1bf142def7d0e0586,fafdcd668e3743c1bb461111dcafc2a4,0,522.0


In [121]:
transcript_new.loc[transcript_new.duplicated(subset = ['person', 'clean_offer_id', 'time_rec'], keep = 'first'), 'time_view'] 

2        624.0
4        624.0
6        624.0
9        522.0
11       522.0
14       504.0
16       504.0
18       348.0
20       348.0
27       510.0
29       510.0
31       348.0
33       348.0
35       690.0
37       690.0
39       408.0
41       408.0
43       360.0
45       360.0
71       360.0
73       360.0
79       186.0
81       186.0
84       168.0
86       168.0
90       648.0
92       648.0
94       168.0
96       168.0
101      342.0
         ...  
86429    606.0
86431    606.0
86457    594.0
86459    594.0
86477    636.0
86479    636.0
86494    588.0
86496    588.0
86507    576.0
86509    576.0
86527    648.0
86529    648.0
86543    612.0
86545    612.0
86578    582.0
86580    582.0
86602    576.0
86604    576.0
86619    576.0
86621    576.0
86623    648.0
86625    648.0
86658    588.0
86660    588.0
86672    672.0
86674    672.0
86749    600.0
86751    600.0
86761    606.0
86763    606.0
Name: time_view, Length: 19044, dtype: float64

In [115]:
transcript_received[transcript_received['person']=='61b2f0a885cf423c83d1395a362ffc01']

,person,clean_offer_id,time
2306,61b2f0a885cf423c83d1395a362ffc01,fafdcd668e3743c1bb461111dcafc2a4,0
55508,61b2f0a885cf423c83d1395a362ffc01,9b98b8c7a33c4b65b9aebfe6a799e6d9,168
113127,61b2f0a885cf423c83d1395a362ffc01,fafdcd668e3743c1bb461111dcafc2a4,336
152919,61b2f0a885cf423c83d1395a362ffc01,ae264e3637204a6fb9bb56bc8210ddfd,408
203877,61b2f0a885cf423c83d1395a362ffc01,3f207df678b143eea3cee63160fa8bed,504
247405,61b2f0a885cf423c83d1395a362ffc01,2906b810c7d4411798c6938adc9daaa5,576


In [116]:
transcript_new[transcript_new['person']=='61b2f0a885cf423c83d1395a362ffc01']

,person,clean_offer_id,time_rec,time_view
4659,61b2f0a885cf423c83d1395a362ffc01,fafdcd668e3743c1bb461111dcafc2a4,0,60.0
4660,61b2f0a885cf423c83d1395a362ffc01,fafdcd668e3743c1bb461111dcafc2a4,336,60.0
28758,61b2f0a885cf423c83d1395a362ffc01,9b98b8c7a33c4b65b9aebfe6a799e6d9,168,204.0
64575,61b2f0a885cf423c83d1395a362ffc01,ae264e3637204a6fb9bb56bc8210ddfd,408,420.0
77862,61b2f0a885cf423c83d1395a362ffc01,3f207df678b143eea3cee63160fa8bed,504,NaN
88297,61b2f0a885cf423c83d1395a362ffc01,2906b810c7d4411798c6938adc9daaa5,576,582.0


In [122]:
view_trans1 = transcript_new[(transcript_new['time_view'] >= transcript_new['time_rec'])]

view_trans2 = transcript_new[np.isnan(transcript_new['time_view'])]

view_trans = pd.concat([view_trans1, view_trans2], axis = 0).sort_index()
view_trans.head(20)

,person,clean_offer_id,time_rec,time_view
0,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,6.0
1,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,6.0
2,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,624.0
4,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,504,624.0
6,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,576,624.0
7,e2127556f4f64592b11af22de27a7932,2906b810c7d4411798c6938adc9daaa5,0,18.0
8,8ec6ce2a7e7949b1bf142def7d0e0586,fafdcd668e3743c1bb461111dcafc2a4,0,12.0
9,8ec6ce2a7e7949b1bf142def7d0e0586,fafdcd668e3743c1bb461111dcafc2a4,0,522.0
11,8ec6ce2a7e7949b1bf142def7d0e0586,fafdcd668e3743c1bb461111dcafc2a4,504,522.0
12,68617ca6246f4fbc85e91a2a49552598,4d5c57ea9a6940dd891ad53e9dbe8da0,0,84.0


In [102]:
view_trans.loc[view_trans.duplicated(subset = ['person', 'clean_offer_id', 'time_view'], 
                                     keep = 'last'), 'time_view'] = float('NaN')

In [123]:
print(len(view_trans))
(view_trans.duplicated(subset = ['person', 'clean_offer_id', 'time_rec'])).sum()

84396


9463

As seen from 

In [75]:
trans_1 = transcript_new[np.all(np.isnan(transcript_new[['time_view', 'time_comp']]), axis = 1)]
trans_2 = transcript_new[(np.isnan(transcript_new['time_view'])) & 
                        (transcript_new['time_comp'] >= transcript_new['time_rec'])]
trans_3 = transcript_new[(np.isnan(transcript_new['time_comp'])) & 
                        (transcript_new['time_view'] >= transcript_new['time_rec'])]
trans_4 = transcript_new[(transcript_new['time_comp'] >= transcript_new['time_view']) & 
                        (transcript_new['time_view'] >= transcript_new['time_rec'])]

transcript_new2 = pd.concat([trans_1, trans_2, trans_3, trans_4])

In [77]:
transcript_new2.sort_index()

,person,clean_offer_id,time_rec,time_view,time_comp
0,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,6.0,132.0
1,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,6.0,NaN
2,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,624.0,NaN
4,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,504,624.0,NaN
6,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,576,624.0,NaN
7,e2127556f4f64592b11af22de27a7932,2906b810c7d4411798c6938adc9daaa5,0,18.0,NaN
8,8ec6ce2a7e7949b1bf142def7d0e0586,fafdcd668e3743c1bb461111dcafc2a4,0,12.0,NaN
9,8ec6ce2a7e7949b1bf142def7d0e0586,fafdcd668e3743c1bb461111dcafc2a4,0,522.0,NaN
11,8ec6ce2a7e7949b1bf142def7d0e0586,fafdcd668e3743c1bb461111dcafc2a4,504,522.0,NaN
12,68617ca6246f4fbc85e91a2a49552598,4d5c57ea9a6940dd891ad53e9dbe8da0,0,84.0,NaN


In [70]:
transcript_new[(np.isnan(transcript_new['time_comp'])) & 
               (transcript_new['time_view'] >= transcript_new['time_rec'])]

,person,clean_offer_id,time_rec,time_view,time_comp
1,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,6.0,NaN
2,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,624.0,NaN
4,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,504,624.0,NaN
6,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,576,624.0,NaN
7,e2127556f4f64592b11af22de27a7932,2906b810c7d4411798c6938adc9daaa5,0,18.0,NaN
8,8ec6ce2a7e7949b1bf142def7d0e0586,fafdcd668e3743c1bb461111dcafc2a4,0,12.0,NaN
9,8ec6ce2a7e7949b1bf142def7d0e0586,fafdcd668e3743c1bb461111dcafc2a4,0,522.0,NaN
11,8ec6ce2a7e7949b1bf142def7d0e0586,fafdcd668e3743c1bb461111dcafc2a4,504,522.0,NaN
12,68617ca6246f4fbc85e91a2a49552598,4d5c57ea9a6940dd891ad53e9dbe8da0,0,84.0,NaN
21,c4863c7985cf408faee930f111475da3,2298d6c36e964ae4a3e7e9706d1fb8c2,0,24.0,NaN


In [71]:
transcript_new[(transcript_new['time_comp'] >= transcript_new['time_view']) & 
               (transcript_new['time_view'] >= transcript_new['time_rec'])]

,person,clean_offer_id,time_rec,time_view,time_comp
0,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,6.0,132.0
13,389bc3fa690240e798340f5a15918d5c,f19421c1d4aa40978ebb69ca19b0e20d,0,0.0,60.0
14,389bc3fa690240e798340f5a15918d5c,f19421c1d4aa40978ebb69ca19b0e20d,0,0.0,600.0
16,389bc3fa690240e798340f5a15918d5c,f19421c1d4aa40978ebb69ca19b0e20d,0,504.0,600.0
20,389bc3fa690240e798340f5a15918d5c,f19421c1d4aa40978ebb69ca19b0e20d,504,504.0,600.0
30,4b0da7e80e5945209a1fdddfe813dbe0,ae264e3637204a6fb9bb56bc8210ddfd,0,102.0,510.0
31,4b0da7e80e5945209a1fdddfe813dbe0,ae264e3637204a6fb9bb56bc8210ddfd,0,510.0,510.0
33,4b0da7e80e5945209a1fdddfe813dbe0,ae264e3637204a6fb9bb56bc8210ddfd,504,510.0,510.0
38,d53717f5400c4e84affdaeda9dd926b3,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,30.0,30.0
39,d53717f5400c4e84affdaeda9dd926b3,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,30.0,624.0
